# Load dataset


In [1]:
from pathlib import Path

base_path = Path(".").absolute().parents[0]

In [2]:
base_path

PosixPath('/home/20014946/Documents/Final_Project_PEM/Final_Project_PEM')

In [3]:
import sys

sys.path.insert(0, str(base_path / "src"))

In [4]:
str(base_path / "src")

'/home/20014946/Documents/Final_Project_PEM/Final_Project_PEM/src'

In [5]:
from google.cloud import bigquery

client = bigquery.Client(project="artefact-taxonomy")


from utils.text_utils import clean_html
from sklearn.model_selection import train_test_split

In [6]:
CUTOFF = 5

# Define the product_id and target / label
id_col = "adeo_product_id"
title_col = "title"
language_col = "lang"
description_raw = "description"
description_col = "description_clean"
label_raw_col = "mirakl_model_code"

In [7]:
import pandas as pd

pd.options.mode.chained_assignment = None  # default='warn'

In [8]:
sql = """
SELECT * FROM `artefact-taxonomy.pem_uc_add_datasets.temp_simplon_2`
"""

df = client.query(sql).to_dataframe()

In [9]:
df.to_csv('../data/full_data.csv', index=False)

In [10]:
df = pd.read_csv('../data/full_data.csv')

## Check the data
- 100 mirakl classes
- with 4 other attributes:
    style: 02419
    color: 01746
    shape: 00562
    battery included: 15344

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882085 entries, 0 to 882084
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   id                 882085 non-null  object
 1   adeo_product_id    882085 non-null  int64 
 2   title              882085 non-null  object
 3   description        882078 non-null  object
 4   lang               882085 non-null  object
 5   attribute_code     882085 non-null  int64 
 6   attribute_label    882085 non-null  object
 7   attribute_type     882085 non-null  object
 8   is_multivalued     882085 non-null  int64 
 9   is_lov             882085 non-null  int64 
 10  attribute_value    882085 non-null  object
 11  lov_code           882085 non-null  int64 
 12  step_model_code    882085 non-null  int64 
 13  mirakl_model_code  882085 non-null  object
 14  data_source        882085 non-null  object
dtypes: int64(6), object(9)
memory usage: 100.9+ MB


### Number of class available

In [12]:
len(df['mirakl_model_code'].unique())

100

In [13]:
df.groupby('attribute_code').count()

,id,adeo_product_id,title,description,lang,attribute_label,attribute_type,is_multivalued,is_lov,attribute_value,lov_code,step_model_code,mirakl_model_code,data_source
attribute_code,,,,,,,,,,,,,,
562,220916,220916,220916,220916,220916,220916,220916,220916,220916,220916,220916,220916,220916,220916
1746,120945,120945,120945,120945,120945,120945,120945,120945,120945,120945,120945,120945,120945,120945
2419,357047,357047,357047,357040,357047,357047,357047,357047,357047,357047,357047,357047,357047,357047
15344,183177,183177,183177,183177,183177,183177,183177,183177,183177,183177,183177,183177,183177,183177


### Check if any null values

In [14]:
df.isnull().sum()

id                   0
adeo_product_id      0
title                0
description          7
lang                 0
attribute_code       0
attribute_label      0
attribute_type       0
is_multivalued       0
is_lov               0
attribute_value      0
lov_code             0
step_model_code      0
mirakl_model_code    0
data_source          0
dtype: int64

In [15]:
df.isnull().values.any()

True

# Data Cleaning

In [16]:
df[description_col] = df[description_raw].copy
df[description_col] = (
    df[description_raw].loc[df[description_raw].notnull()].apply(clean_html)
)
df.fillna("", inplace=True)

In [17]:
df.isnull().values.any()

False

## Preprocessing

### Down sampling to have only 500 samples (mixture of attributes) per class

In [18]:
n = 500
msk = df.groupby(label_raw_col)[label_raw_col].transform('size') >= n
df_cut = pd.concat((df[msk].groupby(label_raw_col).sample(n=n), df[~msk]), ignore_index=True)
print(df_cut.shape)

(50000, 16)


### Get unique id to ensure a single product will only be either in train or test or val (avoid data leaking)

In [31]:
# Get unique id to ensure a single product will only be either in train or test or val 
# To avoid data leaking
# df_unique = df_cut.copy()
# df_unique.drop(columns=[title_col, language_col, description_raw], inplace=True)
df_unique = df_cut[[id_col,label_raw_col]]
df_unique = df_unique.drop_duplicates(subset=id_col, keep="first")

In [32]:
# Calculate the no. of entries and models removed due to sample <= CUTOFF per model
sample_cut_lackofsample = df.shape[0] - df_cut.shape[0]
model_cut_lackofsample = df[label_raw_col].nunique() - df_cut[label_raw_col].nunique()

In [33]:
# Calculate the no. of entries and models removed due to sample <= CUTOFF per model
sample_cut_lackofinfo = df.shape[0] - df_cut.shape[0] - sample_cut_lackofsample
model_cut_lackofinfo = (
    df[label_raw_col].nunique() - df_cut[label_raw_col].nunique() - model_cut_lackofsample
)

In [34]:
# Split the unique adeo_product_id
# To ensure that a unique adeo_product_id exist in only 1 dataset - to avoid data leaking

train_val, test = train_test_split(df_unique, test_size=0.1, stratify=df_unique[label_raw_col])
train, val = train_test_split(train_val, test_size=0.1, stratify=train_val[label_raw_col])

In [35]:
val.columns

Index(['adeo_product_id', 'mirakl_model_code'], dtype='object')

In [36]:
df_cut.columns

Index(['id', 'adeo_product_id', 'title', 'description', 'lang',
       'attribute_code', 'attribute_label', 'attribute_type', 'is_multivalued',
       'is_lov', 'attribute_value', 'lov_code', 'step_model_code',
       'mirakl_model_code', 'data_source', 'description_clean'],
      dtype='object')

## Merge it back to the main df 

In [37]:
del val[label_raw_col]
val = val.merge(df_cut, how="inner", on=id_col)

In [38]:
del train[label_raw_col]
train = train.merge(df_cut, how="inner", on=id_col)

In [39]:
del test[label_raw_col]
test = test.merge(df_cut, how="inner", on=id_col)

## Check for overlapping

In [40]:
import numpy as np


def check_if_overlap(df1, df2, on_column):
    # merge two dataFrames and add indicator column
    all_df = pd.merge(df1, df2, on=on_column, how="left", indicator="exists")
    # add column to show if each row in first DataFrame exists in second
    all_df["exists"] = np.where(all_df.exists == "both", True, False)
    if all_df["exists"].sum() == 0:
        print("These 2 dataframes are not overlapped over ", on_column)
    else:
        print("There are overlapped entries!!")

In [41]:
check_if_overlap(val, train, id_col)

These 2 dataframes are not overlapped over  adeo_product_id


In [42]:
check_if_overlap(test, train, id_col)

These 2 dataframes are not overlapped over  adeo_product_id


In [43]:
check_if_overlap(test, val, id_col)

These 2 dataframes are not overlapped over  adeo_product_id


## Summary

In [44]:
print("============================")
print("Full Dataset")
print("============================")
print("No. entries:", df.shape[0])
print("No. of models (categories / label): ", df[label_raw_col].nunique())

print("No. of entries removed due to num of samples < CUTOFF: ", sample_cut_lackofsample)
print("No. of models removed due to num of samples < CUTOFF: ", model_cut_lackofsample)

print(
    "No. of entries removed due to lack of product information (no title & no description): ",
    sample_cut_lackofinfo,
)
print(
    "No. of models removed due to to lack of product information (no title & no description): ",
    model_cut_lackofinfo,
)

print("No. of entries after cut ", (df_cut.shape[0]))
print("No. of models after cut ", (df_cut[label_raw_col].nunique()))

print("============================")
print("Train Dataset")
print("============================")
print("No. entries:", train.shape[0])
print("No. of models (categories / label): ", train[label_raw_col].nunique())

print("============================")
print("Val Dataset")
print("============================")
print("No. entries:", val.shape[0])
print("No. of models (categories / label): ", val[label_raw_col].nunique())

print("============================")
print("Test Dataset")
print("============================")
print("No. entries:", test.shape[0])
print("No. of models (categories / label): ", test[label_raw_col].nunique())


print("============================")
print("Verify if Dataset is correct")
print("============================")
if train.shape[0] + val.shape[0] + test.shape[0] == df_cut.shape[0]:
    print("The total no. of entries matched!!")
else:
    print("No. of entries matching ERROR!!")

Full Dataset
No. entries: 882085
No. of models (categories / label):  100
No. of entries removed due to num of samples < CUTOFF:  832085
No. of models removed due to num of samples < CUTOFF:  0
No. of entries removed due to lack of product information (no title & no description):  0
No. of models removed due to to lack of product information (no title & no description):  0
No. of entries after cut  50000
No. of models after cut  100
Train Dataset
No. entries: 40501
No. of models (categories / label):  100
Val Dataset
No. entries: 4477
No. of models (categories / label):  100
Test Dataset
No. entries: 5022
No. of models (categories / label):  100
Verify if Dataset is correct
The total no. of entries matched!!


## Save train, val, test split separately

In [48]:
train.to_csv('../data/train.csv', index=False)

In [49]:
val.to_csv('../data/val.csv', index=False)

In [50]:
test.to_csv('../data/test.csv', index=False)